In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install nibabel

In [ ]:
pip install split-folders

In [ ]:
import nibabel as nib

In [ ]:
# TRAIN_DATASET_PATH = 'BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
#VALIDATION_DATASET_PATH = 'BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'
# TRAIN_DATASET_PATH = "../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/"
# path_masks = "../input/brain-tumor-dataset-with-saliency/masks"

# test_image_flair=nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_100/BraTS20_Training_100_flair.nii').get_fdata()
# print(test_image_flair.max())

In [ ]:
pip install MedPy

In [ ]:
pip install --upgrade scipy --user

In [ ]:
"""
All BraTS multimodal scans are available as NIfTI files (.nii.gz) -> commonly used medical imaging format to store brain imagin data obtained using MRI and describe different MRI settings
T1: T1-weighted, native image, sagittal or axial 2D acquisitions, with 1–6 mm slice thickness.
T1ce: T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.
T2: T2-weighted image, axial 2D acquisition, with 2–6 mm slice thickness.
FLAIR: T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2–6 mm slice thickness.
#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.
"""
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import nibabel as nib
import glob
import random
# from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from skimage import io
from scipy import ndimage as nd
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,denoise_wavelet, estimate_sigma)
from medpy.filter.smoothing import anisotropic_diffusion
from scipy.signal import wiener
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
from scipy.signal import convolve2d as conv2
from skimage import color, data, restoration
scaler = MinMaxScaler()








##########################
#This part of the code to get an initial understanding of the dataset.
#################################
#PART 1: Load sample images and visualize
#Includes, dividing each image by its max to scale them to [0,1]
#Converting mask from float to uint8
#Changing mask pixel values (labels) from 4 to 3 (as the original labels are 0, 1, 2, 4)
#Visualize
###########################################
#View a few images

#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.


# Setting up of image path and scaling the image

# TRAIN_DATASET_PATH = 'BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
TRAIN_DATASET_PATH = "../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/"
#VALIDATION_DATASET_PATH = 'BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'


gaussian_cleaned_psnr_flair=[]
gaussian_cleaned_mse_flair=[]
gaussian_cleaned_ssim_flair=[]
gaussian_cleaned_psnr_t1=[]
gaussian_cleaned_mse_t1=[]
gaussian_cleaned_ssim_t1=[]
gaussian_cleaned_psnr_t1ce=[]
gaussian_cleaned_mse_t1ce=[]
gaussian_cleaned_ssim_t1ce=[]
gaussian_cleaned_psnr_t2=[]
gaussian_cleaned_mse_t2=[]
gaussian_cleaned_ssim_t2=[]

bilateral_cleaned_psnr_flair=[]
bilateral_cleaned_mse_flair=[]
bilateral_cleaned_ssim_flair=[]
bilateral_cleaned_psnr_t1=[]
bilateral_cleaned_mse_t1=[]
bilateral_cleaned_ssim_t1=[]
bilateral_cleaned_psnr_t1ce=[]
bilateral_cleaned_mse_t1ce=[]
bilateral_cleaned_ssim_t1ce=[]
bilateral_cleaned_psnr_t2=[]
bilateral_cleaned_mse_t2=[]
bilateral_cleaned_ssim_t2=[]

aniso_cleaned_psnr_flair=[]
aniso_cleaned_mse_flair=[]
aniso_cleaned_ssim_flair=[]
aniso_cleaned_psnr_t1=[]
aniso_cleaned_mse_t1=[]
aniso_cleaned_ssim_t1=[]
aniso_cleaned_psnr_t1ce=[]
aniso_cleaned_mse_t1ce=[]
aniso_cleaned_ssim_t1ce=[]
aniso_cleaned_psnr_t2=[]
aniso_cleaned_mse_t2=[]
aniso_cleaned_ssim_t2=[]

wiener_cleaned_psnr_flair=[]
wiener_cleaned_mse_flair=[]
wiener_cleaned_ssim_flair=[]
wiener_cleaned_psnr_t1=[]
wiener_cleaned_mse_t1=[]
wiener_cleaned_ssim_t1=[]
wiener_cleaned_psnr_t1ce=[]
wiener_cleaned_mse_t1ce=[]
wiener_cleaned_ssim_t1ce=[]
wiener_cleaned_psnr_t2=[]
wiener_cleaned_mse_t2=[]
wiener_cleaned_ssim_t2=[]

m=2
n=4
    
for i in range(m,n):
    if(i<10):
        test_image_flair=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_flair.nii".format(i,i)).get_fdata()
        # print(test_image_flair.max())
        #Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
        test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


        test_image_t1=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_t1.nii".format(i,i)).get_fdata()
        test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

        test_image_t1ce=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_t1ce.nii".format(i,i)).get_fdata()
        test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

        test_image_t2=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_t2.nii".format(i,i)).get_fdata()
        test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

        test_mask=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_seg.nii".format(i,i)).get_fdata()
        test_mask=test_mask.astype(np.uint8)
    #     print("BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i))
    elif(9<i<100):
        test_image_flair=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_flair.nii".format(i,i)).get_fdata()
        # print(test_image_flair.max())
        #Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
        test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


        test_image_t1=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_t1.nii".format(i,i)).get_fdata()
        test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

        test_image_t1ce=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_t1ce.nii".format(i,i)).get_fdata()
        test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

        test_image_t2=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_t2.nii".format(i,i)).get_fdata()
        test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

        test_mask=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_seg.nii".format(i,i)).get_fdata()
        test_mask=test_mask.astype(np.uint8)
    #     print("BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i))
    else:
        test_image_flair=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_flair.nii".format(i,i)).get_fdata()
        # print(test_image_flair.max())
        #Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
        test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


        test_image_t1=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_t1.nii".format(i,i)).get_fdata()
        test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

        test_image_t1ce=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_t1ce.nii".format(i,i)).get_fdata()
        test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

        test_image_t2=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_t2.nii".format(i,i)).get_fdata()
        test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

        test_mask=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i)).get_fdata()
        test_mask=test_mask.astype(np.uint8)
    #     print("BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i))


    
    # img_as_float converts an image from any type to a floating point type in [0, 1], whether it is a color image or not. For example, a (512, 512) image containing uint8 values in the range 0-255 will be converted to a (512, 512) image containing float64 values in the range 0-1.
    #converting the imaage to float type. Since mathematical calculations are required on the image and also mostly, the skimage functions accepts float type image.
    noisy_image_flair=img_as_float(test_image_flair)
    noisy_image_t1=img_as_float(test_image_t1)
    noisy_image_t1ce=img_as_float(test_image_t1ce)
    noisy_image_t2=img_as_float(test_image_t2)
    noisy_mask=img_as_float(test_mask)
    # print(test_image_t1.shape)




    # DENOISING FILTERS

    gaussian_img_flair = nd.gaussian_filter(noisy_image_flair, sigma=5)
    gaussian_img_t1 = nd.gaussian_filter(noisy_image_t1, sigma=5)
    gaussian_img_t1ce = nd.gaussian_filter(noisy_image_t1ce, sigma=5)
    gaussian_img_t2 = nd.gaussian_filter(noisy_image_t2, sigma=5)
    gaussian_mask = nd.gaussian_filter(noisy_mask, sigma=5)
    # print(gaussian_img_t1.shape)



    # print(np.unique(test_mask))  #0, 1, 2, 4 (Need to reencode to 0, 1, 2, 3)
    test_mask[test_mask==4] = 3  #Reassign mask values 4 to 3
    # print(np.unique(test_mask)) 
    n_slice=random.randint(0, test_mask.shape[2])


    bilateral_img_flair = denoise_bilateral(noisy_image_flair[:,:,n_slice],sigma_spatial=15)
    bilateral_img_tl = denoise_bilateral(noisy_image_t1[:,:,n_slice],sigma_spatial=15)
    bilateral_img_t1ce = denoise_bilateral(noisy_image_t1ce[:,:,n_slice],sigma_spatial=15)
    bilateral_img_t2 = denoise_bilateral(noisy_image_t2[:,:,n_slice],sigma_spatial=15)
    bilateral_mask = denoise_bilateral(noisy_mask[:,:,n_slice],sigma_spatial=15)

    img_aniso_filtered_flair = anisotropic_diffusion(noisy_image_flair[:,:,n_slice],niter=50, kappa=20, gamma=0.2, option=1)
    img_aniso_filtered_t1 = anisotropic_diffusion(noisy_image_t1[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)
    img_aniso_filtered_t1ce = anisotropic_diffusion(noisy_image_t1ce[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)
    img_aniso_filtered_t2 = anisotropic_diffusion(noisy_image_t2[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)
    mask_aniso_filtered = anisotropic_diffusion(noisy_mask[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)


    rng = np.random.default_rng()
    psf = np.ones((5, 5)) / 25

    flair = test_image_flair[:,:,n_slice]
    flair = conv2(flair, psf, 'same')
    flair += 0.1 * flair.std() * rng.standard_normal(flair.shape)

    t1 = test_image_t1[:,:,n_slice]
    t1 = conv2(t1, psf, 'same')
    t1 += 0.1 * t1.std() * rng.standard_normal(t1.shape)

    t1ce = test_image_t1ce[:,:,n_slice]
    t1ce = conv2(t1ce, psf, 'same')
    t1ce += 0.1 * t1ce.std() * rng.standard_normal(t1ce.shape)

    t2 = test_image_t2[:,:,n_slice]
    t2 = conv2(t2, psf, 'same')
    t2 += 0.1 * t2.std() * rng.standard_normal(t2.shape)

    mask = test_mask[:,:,n_slice]
    mask = conv2(mask, psf, 'same')
    mask += 0.1 * mask.std() * rng.standard_normal(mask.shape)

    deconvolved_flair, _ = restoration.unsupervised_wiener(flair, psf)
    deconvolved_t1, _ = restoration.unsupervised_wiener(t1, psf)
    deconvolved_t1ce, _ = restoration.unsupervised_wiener(t1ce, psf)
    deconvolved_t2, _ = restoration.unsupervised_wiener(t2, psf)
    deconvolved_mask, _ = restoration.unsupervised_wiener(mask, psf)






    # Gaussian
    gaussian_cleaned_psnr_flair.append(peak_signal_noise_ratio(test_image_flair[:,:,n_slice], gaussian_img_flair[:,:,n_slice]))
    gaussian_cleaned_mse_flair.append(mean_squared_error(test_image_flair[:,:,n_slice], gaussian_img_flair[:,:,n_slice]))
    gaussian_cleaned_ssim_flair.append(ssim(test_image_flair[:,:,n_slice], gaussian_img_flair[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    gaussian_cleaned_psnr_t1.append(peak_signal_noise_ratio(test_image_t1[:,:,n_slice], gaussian_img_t1[:,:,n_slice]))
    gaussian_cleaned_mse_t1.append(mean_squared_error(test_image_t1[:,:,n_slice], gaussian_img_t1[:,:,n_slice]))
    gaussian_cleaned_ssim_t1.append(ssim(test_image_t1[:,:,n_slice], gaussian_img_t1[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    gaussian_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice]))
    gaussian_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice]))
    gaussian_cleaned_ssim_t1ce.append(ssim(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    gaussian_cleaned_psnr_t2.append(peak_signal_noise_ratio(test_image_t2[:,:,n_slice], gaussian_img_t2[:,:,n_slice]))
    gaussian_cleaned_mse_t2.append(mean_squared_error(test_image_t2[:,:,n_slice], gaussian_img_t2[:,:,n_slice]))
    gaussian_cleaned_ssim_t2.append(ssim(test_image_t2[:,:,n_slice], gaussian_img_t2[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    
    #Bilateral
    bilateral_cleaned_psnr_flair.append(peak_signal_noise_ratio(test_image_flair[:,:,n_slice],bilateral_img_flair))
    bilateral_cleaned_mse_flair.append(mean_squared_error(test_image_flair[:,:,n_slice], bilateral_img_flair))
    bilateral_cleaned_ssim_flair.append(ssim(test_image_flair[:,:,n_slice], bilateral_img_flair, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    bilateral_cleaned_psnr_t1.append(peak_signal_noise_ratio(test_image_t1[:,:,n_slice],bilateral_img_tl))
    bilateral_cleaned_mse_t1.append(mean_squared_error(test_image_t1[:,:,n_slice], bilateral_img_tl))
    bilateral_cleaned_ssim_t1.append(ssim(test_image_t1[:,:,n_slice], bilateral_img_tl, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    bilateral_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice],bilateral_img_t1ce))
    bilateral_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce[:,:,n_slice], bilateral_img_t1ce))
    bilateral_cleaned_ssim_t1ce.append(ssim(test_image_t1ce[:,:,n_slice], bilateral_img_t1ce, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    bilateral_cleaned_psnr_t2.append(peak_signal_noise_ratio(test_image_t2[:,:,n_slice],bilateral_img_t2))
    bilateral_cleaned_mse_t2.append(mean_squared_error(test_image_t2[:,:,n_slice], bilateral_img_t2))
    bilateral_cleaned_ssim_t2.append(ssim(test_image_t2[:,:,n_slice], bilateral_img_t2, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    
    #Anisotropic
    aniso_cleaned_psnr_flair.append(peak_signal_noise_ratio(test_image_flair[:,:,n_slice], img_aniso_filtered_flair))
    aniso_cleaned_mse_flair.append(mean_squared_error(test_image_flair[:,:,n_slice], img_aniso_filtered_flair))
    aniso_cleaned_ssim_flair.append(ssim(test_image_flair[:,:,n_slice], img_aniso_filtered_flair, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    aniso_cleaned_psnr_t1.append(peak_signal_noise_ratio(test_image_t1[:,:,n_slice], img_aniso_filtered_t1))
    aniso_cleaned_mse_t1.append(mean_squared_error(test_image_t1[:,:,n_slice], img_aniso_filtered_t1))
    aniso_cleaned_ssim_t1.append(ssim(test_image_t1[:,:,n_slice], img_aniso_filtered_t1, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    aniso_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice], img_aniso_filtered_t1ce))
    aniso_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce[:,:,n_slice], img_aniso_filtered_t1ce))
    aniso_cleaned_ssim_t1ce.append(ssim(test_image_t1ce[:,:,n_slice], img_aniso_filtered_t1ce, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    aniso_cleaned_psnr_t2.append(peak_signal_noise_ratio(test_image_t2[:,:,n_slice], img_aniso_filtered_t2))
    aniso_cleaned_mse_t2.append(mean_squared_error(test_image_t2[:,:,n_slice], img_aniso_filtered_t2))
    aniso_cleaned_ssim_t2.append(ssim(test_image_t2[:,:,n_slice], img_aniso_filtered_t2, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))

    
    #Wiener
    wiener_cleaned_psnr_flair.append(peak_signal_noise_ratio(test_image_flair[:,:,n_slice],deconvolved_flair))
    wiener_cleaned_mse_flair.append(mean_squared_error(test_image_flair[:,:,n_slice], deconvolved_flair))
    wiener_cleaned_ssim_flair.append(ssim(test_image_flair[:,:,n_slice], deconvolved_flair, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    wiener_cleaned_psnr_t1.append(peak_signal_noise_ratio(test_image_t1[:,:,n_slice],deconvolved_t1))
    wiener_cleaned_mse_t1.append(mean_squared_error(test_image_t1[:,:,n_slice], deconvolved_t1))
    wiener_cleaned_ssim_t1.append(ssim(test_image_t1[:,:,n_slice], deconvolved_t1, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    wiener_cleaned_psnr_t1ce.append(peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice],deconvolved_t1ce))
    wiener_cleaned_mse_t1ce.append(mean_squared_error(test_image_t1ce[:,:,n_slice], deconvolved_t1ce))
    wiener_cleaned_ssim_t1ce.append(ssim(test_image_t1ce[:,:,n_slice], deconvolved_t1ce, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))
    
    wiener_cleaned_psnr_t2.append(peak_signal_noise_ratio(test_image_t2[:,:,n_slice],deconvolved_t2))
    wiener_cleaned_mse_t2.append(mean_squared_error(test_image_t2[:,:,n_slice], deconvolved_t2))
    wiener_cleaned_ssim_t2.append(ssim(test_image_t2[:,:,n_slice], deconvolved_t2, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255))





gaussian_sum_psnr_flair=0
gaussian_sum_psnr_t1=0
gaussian_sum_psnr_t1ce=0
gaussian_sum_psnr_t2=0
gaussian_sum_mse_flair=0
gaussian_sum_mse_t1=0
gaussian_sum_mse_t1ce=0
gaussian_sum_mse_t2=0
gaussian_sum_ssim_flair=0
gaussian_sum_ssim_t1=0
gaussian_sum_ssim_t1ce=0
gaussian_sum_ssim_t2=0

bilateral_sum_psnr_flair=0
bilateral_sum_psnr_t1=0
bilateral_sum_psnr_t1ce=0
bilateral_sum_psnr_t2=0
bilateral_sum_mse_flair=0
bilateral_sum_mse_t1=0
bilateral_sum_mse_t1ce=0
bilateral_sum_mse_t2=0
bilateral_sum_ssim_flair=0
bilateral_sum_ssim_t1=0
bilateral_sum_ssim_t1ce=0
bilateral_sum_ssim_t2=0

aniso_sum_psnr_flair=0
aniso_sum_psnr_t1=0
aniso_sum_psnr_t1ce=0
aniso_sum_psnr_t2=0
aniso_sum_mse_flair=0
aniso_sum_mse_t1=0
aniso_sum_mse_t1ce=0
aniso_sum_mse_t2=0
aniso_sum_ssim_flair=0
aniso_sum_ssim_t1=0
aniso_sum_ssim_t1ce=0
aniso_sum_ssim_t2=0

wiener_sum_psnr_flair=0
wiener_sum_psnr_t1=0
wiener_sum_psnr_t1ce=0
wiener_sum_psnr_t2=0
wiener_sum_mse_flair=0
wiener_sum_mse_t1=0
wiener_sum_mse_t1ce=0
wiener_sum_mse_t2=0
wiener_sum_ssim_flair=0
wiener_sum_ssim_t1=0
wiener_sum_ssim_t1ce=0
wiener_sum_ssim_t2=0
for i in range(0,(n-m)):
    gaussian_sum_psnr_flair +=gaussian_cleaned_psnr_flair[i]
    gaussian_sum_psnr_t1 +=gaussian_cleaned_psnr_t1[i]
    gaussian_sum_psnr_t1ce +=gaussian_cleaned_psnr_t1ce[i]
    gaussian_sum_psnr_t2 +=gaussian_cleaned_psnr_t2[i]
    gaussian_sum_mse_flair +=gaussian_cleaned_mse_flair[i]
    gaussian_sum_mse_t1 +=gaussian_cleaned_mse_t1[i]
    gaussian_sum_mse_t1ce +=gaussian_cleaned_mse_t1ce[i]
    gaussian_sum_mse_t2 +=gaussian_cleaned_mse_t2[i]
    gaussian_sum_ssim_flair +=gaussian_cleaned_ssim_flair[i]
    gaussian_sum_ssim_t1 +=gaussian_cleaned_ssim_t1[i]
    gaussian_sum_ssim_t1ce +=gaussian_cleaned_ssim_t1ce[i]
    gaussian_sum_ssim_t2 +=gaussian_cleaned_ssim_t2[i]
    
    bilateral_sum_psnr_flair +=bilateral_cleaned_psnr_flair[i]
    bilateral_sum_psnr_t1 +=bilateral_cleaned_psnr_t1[i]
    bilateral_sum_psnr_t1ce +=bilateral_cleaned_psnr_t1ce[i]
    bilateral_sum_psnr_t2 +=bilateral_cleaned_psnr_t2[i]
    bilateral_sum_mse_flair +=bilateral_cleaned_mse_flair[i]
    bilateral_sum_mse_t1 +=bilateral_cleaned_mse_t1[i]
    bilateral_sum_mse_t1ce +=bilateral_cleaned_mse_t1ce[i]
    bilateral_sum_mse_t2 +=bilateral_cleaned_mse_t2[i]
    bilateral_sum_ssim_flair +=bilateral_cleaned_ssim_flair[i]
    bilateral_sum_ssim_t1 +=bilateral_cleaned_ssim_t1[i]
    bilateral_sum_ssim_t1ce +=bilateral_cleaned_ssim_t1ce[i]
    bilateral_sum_ssim_t2 +=bilateral_cleaned_ssim_t2[i]
    
    aniso_sum_psnr_flair +=aniso_cleaned_psnr_flair[i]
    aniso_sum_psnr_t1 +=aniso_cleaned_psnr_t1[i]
    aniso_sum_psnr_t1ce +=aniso_cleaned_psnr_t1ce[i]
    aniso_sum_psnr_t2 +=aniso_cleaned_psnr_t2[i]
    aniso_sum_mse_flair +=aniso_cleaned_mse_flair[i]
    aniso_sum_mse_t1 +=aniso_cleaned_mse_t1[i]
    aniso_sum_mse_t1ce +=aniso_cleaned_mse_t1ce[i]
    aniso_sum_mse_t2 +=aniso_cleaned_mse_t2[i]
    aniso_sum_ssim_flair +=aniso_cleaned_ssim_flair[i]
    aniso_sum_ssim_t1 +=aniso_cleaned_ssim_t1[i]
    aniso_sum_ssim_t1ce +=aniso_cleaned_ssim_t1ce[i]
    aniso_sum_ssim_t2 +=aniso_cleaned_ssim_t2[i]
    
    wiener_sum_psnr_flair +=wiener_cleaned_psnr_flair[i]
    wiener_sum_psnr_t1 +=wiener_cleaned_psnr_t1[i]
    wiener_sum_psnr_t1ce +=wiener_cleaned_psnr_t1ce[i]
    wiener_sum_psnr_t2 +=wiener_cleaned_psnr_t2[i]
    wiener_sum_mse_flair +=wiener_cleaned_mse_flair[i]
    wiener_sum_mse_t1 +=wiener_cleaned_mse_t1[i]
    wiener_sum_mse_t1ce +=wiener_cleaned_mse_t1ce[i]
    wiener_sum_mse_t2 +=wiener_cleaned_mse_t2[i]
    wiener_sum_ssim_flair +=wiener_cleaned_ssim_flair[i]
    wiener_sum_ssim_t1 +=wiener_cleaned_ssim_t1[i]
    wiener_sum_ssim_t1ce +=wiener_cleaned_ssim_t1ce[i]
    wiener_sum_ssim_t2 +=wiener_cleaned_ssim_t2[i]
    
print("-"*10 + "FLAIR" + "-"*10 + "\n"*2)
print(" "*10 + "GAUSSIAN" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(gaussian_sum_psnr_flair/(n-m)))
print(" "*5 + "mse: {}".format(gaussian_sum_mse_flair/(n-m)))
print(" "*5 + "ssim: {}".format(gaussian_sum_ssim_flair/(n-m)))

print("\n"*2+ " "*10 + "BILATERAL" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(bilateral_sum_psnr_flair/(n-m)))
print(" "*5 + "mse: {}".format(bilateral_sum_mse_flair/(n-m)))
print(" "*5 + "ssim: {}".format(bilateral_sum_ssim_flair/(n-m)))

print("\n"*2 + " "*10 + "ANISOTROPIC" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(aniso_sum_psnr_flair/(n-m)))
print(" "*5 + "mse: {}".format(aniso_sum_mse_flair/(n-m)))
print(" "*5 + "ssim: {}".format(aniso_sum_ssim_flair/(n-m)))

print("\n"*2 + " "*10 + "WIENER" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(wiener_sum_psnr_flair/(n-m)))
print(" "*5 + "mse: {}".format(wiener_sum_mse_flair/(n-m)))
print(" "*5 + "ssim: {}".format(wiener_sum_ssim_flair/(n-m)))

print("\n"*2)


print("-"*10 + "T1" + "-"*10 + "\n"*2)
print(" "*10 + "GAUSSIAN" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(gaussian_sum_psnr_t1/(n-m)))
print(" "*5 + "mse: {}".format(gaussian_sum_mse_t1/(n-m)))
print(" "*5 + "ssim: {}".format(gaussian_sum_ssim_t1/(n-m)))

print("\n"*2 + " "*10 + "BILATERAL" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(bilateral_sum_psnr_t1/(n-m)))
print(" "*5 + "mse: {}".format(bilateral_sum_mse_t1/(n-m)))
print(" "*5 + "ssim: {}".format(bilateral_sum_ssim_t1/(n-m)))

print("\n"*2 + " "*10 + "ANISOTROPIC" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(aniso_sum_psnr_t1/(n-m)))
print(" "*5 + "mse: {}".format(aniso_sum_mse_t1/(n-m)))
print(" "*5 + "ssim: {}".format(aniso_sum_ssim_t1/(n-m)))

print("\n"*2 + " "*10 + "WIENER" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(wiener_sum_psnr_t1/(n-m)))
print(" "*5 + "mse: {}".format(wiener_sum_mse_t1/(n-m)))
print(" "*5 + "ssim: {}".format(wiener_sum_ssim_t1/(n-m)))

print("\n"*2)


print("-"*10 + "T1CE" + "-"*10 + "\n"*2)
print(" "*10 + "GAUSSIAN" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(gaussian_sum_psnr_t1ce/(n-m)))
print(" "*5 + "mse: {}".format(gaussian_sum_mse_t1ce/(n-m)))
print(" "*5 + "ssim: {}".format(gaussian_sum_ssim_t1ce/(n-m)))

print("\n"*2 + " "*10 + "BILATERAL" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(bilateral_sum_psnr_t1ce/(n-m)))
print(" "*5 + "mse: {}".format(bilateral_sum_mse_t1ce/(n-m)))
print(" "*5 + "ssim: {}".format(bilateral_sum_ssim_t1ce/(n-m)))

print("\n"*2 + " "*10 + "ANISOTROPIC" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(aniso_sum_psnr_t1ce/(n-m)))
print(" "*5 + "mse: {}".format(aniso_sum_mse_t1ce/(n-m)))
print(" "*5 + "ssim: {}".format(aniso_sum_ssim_t1ce/(n-m)))

print("\n"*2 + " "*10 + "WIENER" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(wiener_sum_psnr_t1ce/(n-m)))
print(" "*5 + "mse: {}".format(wiener_sum_mse_t1ce/(n-m)))
print(" "*5 + "ssim: {}".format(wiener_sum_ssim_t1ce/(n-m)))

print("\n"*2)

print("-"*10 + "T2" + "-"*10 + "\n"*2)
print(" "*10 + "GAUSSIAN" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(gaussian_sum_psnr_t2/(n-m)))
print(" "*5 + "mse: {}".format(gaussian_sum_mse_t2/(n-m)))
print(" "*5 + "ssim: {}".format(gaussian_sum_ssim_t2/(n-m)))

print("\n"*2 + " "*10 + "BILATERAL" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(bilateral_sum_psnr_t2/(n-m)))
print(" "*5 + "mse: {}".format(bilateral_sum_mse_t2/(n-m)))
print(" "*5 + "ssim: {}".format(bilateral_sum_ssim_t2/(n-m)))

print("\n"*2 + " "*10 + "ANISOTROPIC" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(aniso_sum_psnr_t2/(n-m)))
print(" "*5 + "mse: {}".format(aniso_sum_mse_t2/(n-m)))
print(" "*5 + "ssim: {}".format(aniso_sum_ssim_t2/(n-m)))

print("\n"*2 + " "*10 + "WIENER" + " "*10 + "\n")
print(" "*5 + "psnr: {}".format(wiener_sum_psnr_t2/(n-m)))
print(" "*5 + "mse: {}".format(wiener_sum_mse_t2/(n-m)))
print(" "*5 + "ssim: {}".format(wiener_sum_ssim_t2/(n-m)))

print("\n"*2)


In [ ]:
"""
All BraTS multimodal scans are available as NIfTI files (.nii.gz) -> commonly used medical imaging format to store brain imagin data obtained using MRI and describe different MRI settings
T1: T1-weighted, native image, sagittal or axial 2D acquisitions, with 1–6 mm slice thickness.
T1ce: T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.
T2: T2-weighted image, axial 2D acquisition, with 2–6 mm slice thickness.
FLAIR: T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2–6 mm slice thickness.
#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.
"""
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import nibabel as nib
import glob
import random
# from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from skimage import io
from scipy import ndimage as nd
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,denoise_wavelet, estimate_sigma)
from medpy.filter.smoothing import anisotropic_diffusion
from scipy.signal import wiener
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
scaler = MinMaxScaler()








##########################
#This part of the code to get an initial understanding of the dataset.
#################################
#PART 1: Load sample images and visualize
#Includes, dividing each image by its max to scale them to [0,1]
#Converting mask from float to uint8
#Changing mask pixel values (labels) from 4 to 3 (as the original labels are 0, 1, 2, 4)
#Visualize
###########################################
#View a few images

#Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.


# Setting up of image path and scaling the image
m=9

if(m<10):
        test_image_flair=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_flair.nii".format(i,i)).get_fdata()
        # print(test_image_flair.max())
        #Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
        test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


        test_image_t1=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_t1.nii".format(i,i)).get_fdata()
        test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

        test_image_t1ce=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_t1ce.nii".format(i,i)).get_fdata()
        test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

        test_image_t2=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_t2.nii".format(i,i)).get_fdata()
        test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

        test_mask=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_00{}/BraTS20_Training_00{}_seg.nii".format(i,i)).get_fdata()
        test_mask=test_mask.astype(np.uint8)
    #     print("BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i))
elif(9<m<100):
        test_image_flair=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_flair.nii".format(i,i)).get_fdata()
        # print(test_image_flair.max())
        #Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
        test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


        test_image_t1=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_t1.nii".format(i,i)).get_fdata()
        test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

        test_image_t1ce=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_t1ce.nii".format(i,i)).get_fdata()
        test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

        test_image_t2=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_t2.nii".format(i,i)).get_fdata()
        test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

        test_mask=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_0{}/BraTS20_Training_0{}_seg.nii".format(i,i)).get_fdata()
        test_mask=test_mask.astype(np.uint8)
    #     print("BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i))
else:
        test_image_flair=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_flair.nii".format(i,i)).get_fdata()
        # print(test_image_flair.max())
        #Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
        test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


        test_image_t1=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_t1.nii".format(i,i)).get_fdata()
        test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

        test_image_t1ce=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_t1ce.nii".format(i,i)).get_fdata()
        test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

        test_image_t2=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_t2.nii".format(i,i)).get_fdata()
        test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

        test_mask=nib.load(TRAIN_DATASET_PATH + "BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i)).get_fdata()
        test_mask=test_mask.astype(np.uint8)
    #     print("BraTS20_Training_{}/BraTS20_Training_{}_seg.nii".format(i,i))






# img_as_float converts an image from any type to a floating point type in [0, 1], whether it is a color image or not. For example, a (512, 512) image containing uint8 values in the range 0-255 will be converted to a (512, 512) image containing float64 values in the range 0-1.
#converting the imaage to float type. Since mathematical calculations are required on the image and also mostly, the skimage functions accepts float type image.
noisy_image_flair=img_as_float(test_image_flair)
noisy_image_t1=img_as_float(test_image_t1)
noisy_image_t1ce=img_as_float(test_image_t1ce)
noisy_image_t2=img_as_float(test_image_t2)
noisy_mask=img_as_float(test_mask)
# print(test_image_t1.shape)




# DENOISING FILTERS

gaussian_img_flair = nd.gaussian_filter(noisy_image_flair, sigma=5)
gaussian_img_t1 = nd.gaussian_filter(noisy_image_t1, sigma=5)
gaussian_img_t1ce = nd.gaussian_filter(noisy_image_t1ce, sigma=5)
gaussian_img_t2 = nd.gaussian_filter(noisy_image_t2, sigma=5)
gaussian_mask = nd.gaussian_filter(noisy_mask, sigma=5)
# print(gaussian_img_t1.shape)



# print(np.unique(test_mask))  #0, 1, 2, 4 (Need to reencode to 0, 1, 2, 3)
test_mask[test_mask==4] = 3  #Reassign mask values 4 to 3
# print(np.unique(test_mask)) 
n_slice=random.randint(0, test_mask.shape[2])


bilateral_img_flair = denoise_bilateral(noisy_image_flair[:,:,n_slice],sigma_spatial=15)
bilateral_img_tl = denoise_bilateral(noisy_image_t1[:,:,n_slice],sigma_spatial=15)
bilateral_img_t1ce = denoise_bilateral(noisy_image_t1ce[:,:,n_slice],sigma_spatial=15)
bilateral_img_t2 = denoise_bilateral(noisy_image_t2[:,:,n_slice],sigma_spatial=15)
bilateral_mask = denoise_bilateral(noisy_mask[:,:,n_slice],sigma_spatial=15)

img_aniso_filtered_flair = anisotropic_diffusion(noisy_image_flair[:,:,n_slice],niter=50, kappa=20, gamma=0.2, option=1)
img_aniso_filtered_t1 = anisotropic_diffusion(noisy_image_t1[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)
img_aniso_filtered_t1ce = anisotropic_diffusion(noisy_image_t1ce[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)
img_aniso_filtered_t2 = anisotropic_diffusion(noisy_image_t2[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)
mask_aniso_filtered = anisotropic_diffusion(noisy_mask[:,:,n_slice], niter=50, kappa=20, gamma=0.2, option=1)


rng = np.random.default_rng()
psf = np.ones((5, 5)) / 25

flair = test_image_flair[:,:,n_slice]
flair = conv2(flair, psf, 'same')
flair += 0.1 * flair.std() * rng.standard_normal(flair.shape)

t1 = test_image_t1[:,:,n_slice]
t1 = conv2(t1, psf, 'same')
t1 += 0.1 * t1.std() * rng.standard_normal(t1.shape)

t1ce = test_image_t1ce[:,:,n_slice]
t1ce = conv2(t1ce, psf, 'same')
t1ce += 0.1 * t1ce.std() * rng.standard_normal(t1ce.shape)

t2 = test_image_t2[:,:,n_slice]
t2 = conv2(t2, psf, 'same')
t2 += 0.1 * t2.std() * rng.standard_normal(t2.shape)

mask = test_mask[:,:,n_slice]
mask = conv2(mask, psf, 'same')
mask += 0.1 * mask.std() * rng.standard_normal(mask.shape)

deconvolved_flair, _ = restoration.unsupervised_wiener(flair, psf)
deconvolved_t1, _ = restoration.unsupervised_wiener(t1, psf)
deconvolved_t1ce, _ = restoration.unsupervised_wiener(t1ce, psf)
deconvolved_t2, _ = restoration.unsupervised_wiener(t2, psf)
deconvolved_mask, _ = restoration.unsupervised_wiener(mask, psf)








# Gaussian
plt.figure(figsize=(12, 8))

plt.subplot(231)
plt.imshow(gaussian_img_flair[:,:,n_slice], cmap='gray')
gaussian_cleaned_psnr = peak_signal_noise_ratio(test_image_flair[:,:,n_slice], gaussian_img_flair[:,:,n_slice])
gaussian_cleaned_mse = mean_squared_error(test_image_flair[:,:,n_slice], gaussian_img_flair[:,:,n_slice])
gaussian_cleaned_ssim= ssim(test_image_flair[:,:,n_slice], gaussian_img_flair[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("-"*10 + "GAUSSIAN" + "-"*10 + "\n"*2)
print("PSNR of cleaned image = ", gaussian_cleaned_psnr)
print("MSE of cleaned image = ", gaussian_cleaned_mse)
print("SSIM of cleaned image = ", gaussian_cleaned_ssim)
print("\n"*2)

plt.subplot(232)
plt.imshow(gaussian_img_t1[:,:,n_slice], cmap='gray')
gaussian_cleaned_psnr = peak_signal_noise_ratio(test_image_t1[:,:,n_slice], gaussian_img_t1[:,:,n_slice])
gaussian_cleaned_mse = mean_squared_error(test_image_t1[:,:,n_slice], gaussian_img_t1[:,:,n_slice])
gaussian_cleaned_ssim= ssim(test_image_t1[:,:,n_slice], gaussian_img_t1[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", gaussian_cleaned_psnr)
print("MSE of cleaned image = ", gaussian_cleaned_mse)
print("SSIM of cleaned image = ", gaussian_cleaned_ssim)
print("\n"*2)

plt.subplot(233)
plt.imshow(gaussian_img_t1ce[:,:,n_slice], cmap='gray')
gaussian_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice])
gaussian_cleaned_mse = mean_squared_error(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice])
gaussian_cleaned_ssim= ssim(test_image_t1ce[:,:,n_slice], gaussian_img_t1ce[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", gaussian_cleaned_psnr)
print("MSE of cleaned image = ", gaussian_cleaned_mse)
print("SSIM of cleaned image = ", gaussian_cleaned_ssim)
print("\n"*2)

plt.subplot(234)
plt.imshow(gaussian_img_t2[:,:,n_slice], cmap='gray')
gaussian_cleaned_psnr = peak_signal_noise_ratio(test_image_t2[:,:,n_slice], gaussian_img_t2[:,:,n_slice])
gaussian_cleaned_mse = mean_squared_error(test_image_t2[:,:,n_slice], gaussian_img_t2[:,:,n_slice])
gaussian_cleaned_ssim= ssim(test_image_t2[:,:,n_slice], gaussian_img_t2[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", gaussian_cleaned_psnr)
print("MSE of cleaned image = ", gaussian_cleaned_mse)
print("SSIM of cleaned image = ", gaussian_cleaned_ssim)
print("\n"*2)

plt.subplot(235)
plt.imshow(gaussian_mask[:,:,n_slice])
gaussian_cleaned_psnr = peak_signal_noise_ratio(test_mask[:,:,n_slice], gaussian_mask[:,:,n_slice])
gaussian_cleaned_mse = mean_squared_error(test_mask[:,:,n_slice], gaussian_mask[:,:,n_slice])
gaussian_cleaned_ssim= ssim(test_mask[:,:,n_slice], gaussian_mask[:,:,n_slice], multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", gaussian_cleaned_psnr)
print("MSE of cleaned image = ", gaussian_cleaned_mse)
print("SSIM of cleaned mask = ", gaussian_cleaned_ssim)
print("\n"*2)

plt.show()







# Bilateral

plt.figure(figsize=(12, 8))

plt.subplot(231)
plt.imshow(bilateral_img_flair, cmap='gray')
bilateral_cleaned_psnr = peak_signal_noise_ratio(test_image_flair[:,:,n_slice],bilateral_img_flair)
bilateral_cleaned_mse = mean_squared_error(test_image_flair[:,:,n_slice], bilateral_img_flair)
bilateral_cleaned_ssim= ssim(test_image_flair[:,:,n_slice], bilateral_img_flair, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("-"*10 + "BILATERAL" + "-"*10 + "\n"*2)
print("PSNR of cleaned image = ", bilateral_cleaned_psnr)
print("MSE of cleaned image = ", bilateral_cleaned_mse)
print("SSIM of cleaned image = ", bilateral_cleaned_ssim)
print("\n"*2)

plt.subplot(232)
plt.imshow(bilateral_img_tl, cmap='gray')
bilateral_cleaned_psnr = peak_signal_noise_ratio(test_image_t1[:,:,n_slice],bilateral_img_tl)
bilateral_cleaned_mse = mean_squared_error(test_image_t1[:,:,n_slice], bilateral_img_tl)
bilateral_cleaned_ssim= ssim(test_image_t1[:,:,n_slice], bilateral_img_tl, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", bilateral_cleaned_psnr)
print("MSE of cleaned image = ", bilateral_cleaned_mse)
print("SSIM of cleaned image = ", bilateral_cleaned_ssim)
print("\n"*2)

plt.subplot(233)
plt.imshow(bilateral_img_t1ce, cmap='gray')
bilateral_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice],bilateral_img_t1ce)
bilateral_cleaned_mse = mean_squared_error(test_image_t1ce[:,:,n_slice], bilateral_img_t1ce)
bilateral_cleaned_ssim= ssim(test_image_t1ce[:,:,n_slice], bilateral_img_t1ce, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", bilateral_cleaned_psnr)
print("MSE of cleaned image = ", bilateral_cleaned_mse)
print("SSIM of cleaned image = ", bilateral_cleaned_ssim)
print("\n"*2)

plt.subplot(234)
plt.imshow(bilateral_img_t2, cmap='gray')
bilateral_cleaned_psnr = peak_signal_noise_ratio(test_image_t2[:,:,n_slice],bilateral_img_t2)
bilateral_cleaned_mse = mean_squared_error(test_image_t2[:,:,n_slice], bilateral_img_t2)
bilateral_cleaned_ssim= ssim(test_image_t2[:,:,n_slice], bilateral_img_t2, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", bilateral_cleaned_psnr)
print("MSE of cleaned image = ", bilateral_cleaned_mse)
print("SSIM of cleaned image = ", bilateral_cleaned_ssim)
print("\n"*2)

plt.subplot(235)
plt.imshow(bilateral_mask)
bilateral_cleaned_psnr = peak_signal_noise_ratio(test_mask[:,:,n_slice],bilateral_mask)
bilateral_cleaned_mse = mean_squared_error(test_mask[:,:,n_slice], bilateral_mask)
bilateral_cleaned_ssim= ssim(test_mask[:,:,n_slice], bilateral_mask, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", bilateral_cleaned_psnr)
print("MSE of cleaned image = ", bilateral_cleaned_mse)
print("SSIM of cleaned mask = ", bilateral_cleaned_ssim)
print("\n"*2)

plt.show()








#Anistropic

plt.figure(figsize=(12, 8))

plt.subplot(231)
plt.imshow(img_aniso_filtered_flair, cmap='gray')
aniso_cleaned_psnr = peak_signal_noise_ratio(test_image_flair[:,:,n_slice], img_aniso_filtered_flair)
aniso_cleaned_mse = mean_squared_error(test_image_flair[:,:,n_slice], img_aniso_filtered_flair)
aniso_cleaned_ssim= ssim(test_image_flair[:,:,n_slice], img_aniso_filtered_flair, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("-"*10 + "ANISOTROPIC" + "-"*10 + "\n"*2)
print("PSNR of cleaned image = ", aniso_cleaned_psnr)
print("MSE of cleaned image = ", aniso_cleaned_mse)
print("SSIM of cleaned image = ", aniso_cleaned_ssim)
print("\n"*2)

plt.subplot(232)
plt.imshow(img_aniso_filtered_t1, cmap='gray')
aniso_cleaned_psnr = peak_signal_noise_ratio(test_image_t1[:,:,n_slice], img_aniso_filtered_t1)
aniso_cleaned_mse = mean_squared_error(test_image_t1[:,:,n_slice], img_aniso_filtered_t1)
aniso_cleaned_ssim= ssim(test_image_t1[:,:,n_slice], img_aniso_filtered_t1, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", aniso_cleaned_psnr)
print("MSE of cleaned image = ", aniso_cleaned_mse)
print("SSIM of cleaned image = ", aniso_cleaned_ssim)
print("\n"*2)

plt.subplot(233)
plt.imshow(img_aniso_filtered_t1ce, cmap='gray')
aniso_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice], img_aniso_filtered_t1ce)
aniso_cleaned_mse = mean_squared_error(test_image_t1ce[:,:,n_slice], img_aniso_filtered_t1ce)
aniso_cleaned_ssim= ssim(test_image_t1ce[:,:,n_slice], img_aniso_filtered_t1ce, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", aniso_cleaned_psnr)
print("MSE of cleaned image = ", aniso_cleaned_mse)
print("SSIM of cleaned image = ", aniso_cleaned_ssim)
print("\n"*2)

plt.subplot(234)
plt.imshow(img_aniso_filtered_t2, cmap='gray')
aniso_cleaned_psnr = peak_signal_noise_ratio(test_image_t2[:,:,n_slice], img_aniso_filtered_t2)
aniso_cleaned_mse = mean_squared_error(test_image_t2[:,:,n_slice], img_aniso_filtered_t2)
aniso_cleaned_ssim= ssim(test_image_t2[:,:,n_slice], img_aniso_filtered_t2, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", aniso_cleaned_psnr)
print("MSE of cleaned image = ", aniso_cleaned_mse)
print("SSIM of cleaned image = ", aniso_cleaned_ssim)
print("\n"*2)

plt.subplot(235)
plt.imshow(mask_aniso_filtered)
aniso_cleaned_psnr = peak_signal_noise_ratio(test_mask[:,:,n_slice], mask_aniso_filtered)
aniso_cleaned_mse = mean_squared_error(test_mask[:,:,n_slice], mask_aniso_filtered)
aniso_cleaned_ssim= ssim(test_mask[:,:,n_slice], mask_aniso_filtered, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", aniso_cleaned_psnr)
print("MSE of cleaned image = ", aniso_cleaned_mse)
print("SSIM of cleaned mask = ", aniso_cleaned_ssim)
print("\n"*2)

plt.show()









# Wiener

plt.figure(figsize=(12, 8))

plt.subplot(231)
plt.imshow(deconvolved_flair, cmap='gray')
wiener_cleaned_psnr = peak_signal_noise_ratio(test_image_flair[:,:,n_slice],deconvolved_flair)
wiener_cleaned_mse = mean_squared_error(test_image_flair[:,:,n_slice], deconvolved_flair)
wiener_cleaned_ssim= ssim(test_image_flair[:,:,n_slice], deconvolved_flair, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("-"*10 + "WIENER" + "-"*10 + "\n"*2)
print("PSNR of cleaned image = ", wiener_cleaned_psnr)
print("MSE of cleaned image = ", wiener_cleaned_mse)
print("SSIM of cleaned image = ", wiener_cleaned_ssim)
print("\n"*2)

plt.subplot(232)
plt.imshow(deconvolved_t1, cmap='gray')
wiener_cleaned_psnr = peak_signal_noise_ratio(test_image_t1[:,:,n_slice],deconvolved_t1)
wiener_cleaned_mse = mean_squared_error(test_image_t1[:,:,n_slice], deconvolved_t1)
wiener_cleaned_ssim= ssim(test_image_t1[:,:,n_slice], deconvolved_t1, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", wiener_cleaned_psnr)
print("MSE of cleaned image = ", wiener_cleaned_mse)
print("SSIM of cleaned image = ", wiener_cleaned_ssim)
print("\n"*2)

plt.subplot(233)
plt.imshow(deconvolved_t1ce, cmap='gray')
wiener_cleaned_psnr = peak_signal_noise_ratio(test_image_t1ce[:,:,n_slice],deconvolved_t1ce)
wiener_cleaned_mse = mean_squared_error(test_image_t1ce[:,:,n_slice], deconvolved_t1ce)
wiener_cleaned_ssim= ssim(test_image_t1ce[:,:,n_slice], deconvolved_t1ce, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", wiener_cleaned_psnr)
print("MSE of cleaned image = ", wiener_cleaned_mse)
print("SSIM of cleaned image = ", wiener_cleaned_ssim)
print("\n"*2)

plt.subplot(234)
plt.imshow(deconvolved_t2, cmap='gray')
wiener_cleaned_psnr = peak_signal_noise_ratio(test_image_t2[:,:,n_slice],deconvolved_t2)
wiener_cleaned_mse = mean_squared_error(test_image_t2[:,:,n_slice], deconvolved_t2)
wiener_cleaned_ssim= ssim(test_image_t2[:,:,n_slice], deconvolved_t2, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", wiener_cleaned_psnr)
print("MSE of cleaned image = ", wiener_cleaned_mse)
print("SSIM of cleaned image = ", wiener_cleaned_ssim)
print("\n"*2)

plt.subplot(235)
plt.imshow(deconvolved_mask)
wiener_cleaned_psnr = peak_signal_noise_ratio(test_mask[:,:,n_slice],deconvolved_mask)
wiener_cleaned_mse = mean_squared_error(test_mask[:,:,n_slice], deconvolved_mask)
wiener_cleaned_ssim= ssim(test_mask[:,:,n_slice], deconvolved_mask, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=255)
print("PSNR of cleaned image = ", wiener_cleaned_psnr)
print("MSE of cleaned image = ", wiener_cleaned_mse)
print("SSIM of cleaned mask = ", wiener_cleaned_ssim)
print("\n"*2)

plt.show()




# Raw Images

print("\n"*4)
print("RAW IMAGES")
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.imshow(test_image_flair[:,:,n_slice], cmap='gray')
plt.title('Image flair')
plt.subplot(232)
plt.imshow(test_image_t1[:,:,n_slice], cmap='gray')
plt.title('Image t1')
plt.subplot(233)
plt.imshow(test_image_t1ce[:,:,n_slice], cmap='gray')
plt.title('Image t1ce')
plt.subplot(234)
plt.imshow(test_image_t2[:,:,n_slice], cmap='gray')
plt.title('Image t2')
plt.subplot(235)
plt.imshow(test_mask[:,:,n_slice])
plt.title('Mask')
plt.show()



##################################################
#PART 2: Explore the process of combining images to channels and divide them to patches
#Includes...
#Combining all 4 images to 4 channels of a numpy array.
#
################################################
#Flair, T1CE, annd T2 have the most information
#Combine t1ce, t2, and flair into single multichannel image

combined_x = np.stack([test_image_flair, test_image_t1ce, test_image_t2], axis=3)

#Crop to a size to be divisible by 64 so we can later extract 64x64x64 patches. 
#cropping x, y, and z
#combined_x=combined_x[24:216, 24:216, 13:141]

combined_x=combined_x[56:184, 56:184, 13:141] #Crop to 128x128x128x4

#Do the same for mask
test_mask = test_mask[56:184, 56:184, 13:141]

n_slice=random.randint(0, test_mask.shape[2])
print("\n"*4)
print("CROPPED IMAGES")
plt.figure(figsize=(12, 8))
plt.title("CROPPED IMAGES")
plt.subplot(221)
plt.imshow(combined_x[:,:,n_slice, 0], cmap='gray')
plt.title('Image flair')
plt.subplot(222)
plt.imshow(combined_x[:,:,n_slice, 1], cmap='gray')
plt.title('Image t1ce')
plt.subplot(223)
plt.imshow(combined_x[:,:,n_slice, 2], cmap='gray')
plt.title('Image t2')
plt.subplot(224)
plt.imshow(test_mask[:,:,n_slice])
plt.title('Mask')
plt.show()